In [1]:
from module import models
from module import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

/home/yegekucuk/venv/bitirme/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset

df = datasets.amazon()
df

,name,main_category
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances
...,...,...
1103165,Adidas Regular Fit Men's Track Tops,sports & fitness
1103166,Redwolf Noice Toit Smort - Hoodie (Black),sports & fitness
1103167,Redwolf Schrute Farms B&B - Hoodie (Navy Blue),sports & fitness
1103168,Puma Men Shorts,sports & fitness


In [3]:
# BoW
cv = CountVectorizer()
X = cv.fit_transform(df["name"])
# Label Encoding
le = LabelEncoder()
y = le.fit_transform(df["main_category"])
# Model
lr = models.LogisticRegression()
lr = models.train_and_eval(X, y, classes=le.classes_, model=lr)

                         precision    recall  f1-score   support

            accessories       0.89      0.96      0.92     46242
             appliances       0.96      0.98      0.97     13209
         bags & luggage       0.76      0.53      0.62      4248
        beauty & health       0.90      0.92      0.91      4023
        car & motorbike       0.92      0.89      0.90      2818
grocery & gourmet foods       0.92      0.94      0.93      1337
         home & kitchen       0.87      0.84      0.86      5774
    industrial supplies       0.83      0.78      0.80      1624
          kids' fashion       0.84      0.75      0.79      5367
         men's clothing       0.96      0.98      0.97     30696
            men's shoes       0.89      0.95      0.92     23077
                  music       0.89      0.82      0.85       400
           pet supplies       0.97      0.93      0.95       646
       sports & fitness       0.78      0.68      0.73      4969
                 stores 

In [4]:
models.save_model(lr, "lr_full.pkl")